#### Module 1 Homework - NY Taxi Data Questions

In [3]:
# Imports
import pandas as pd

In [4]:
# Input parameters
green_taxi_trips_2025_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet'
zone_dataset_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv'


In [5]:
# Load Green Taxi Trips Data

green_taxi_trips_2025 = pd.read_parquet(green_taxi_trips_2025_url)
green_taxi_trips_2025.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


#### Question 3

In [6]:
# Check dates
print('Date range:', green_taxi_trips_2025['lpep_pickup_datetime'].min(), 'to', green_taxi_trips_2025['lpep_pickup_datetime'].max())

# Trim dataset
november_2025_trips = green_taxi_trips_2025[
    (green_taxi_trips_2025['lpep_pickup_datetime'] >= '2025-11-01') &
    (green_taxi_trips_2025['lpep_pickup_datetime'] < '2025-12-01')
]

print('Adjusted date range:', november_2025_trips['lpep_pickup_datetime'].min(), 'to', november_2025_trips['lpep_pickup_datetime'].max())

november_2025_trips_leq_1_mile = november_2025_trips[november_2025_trips['trip_distance'] <= 1.0]

print('November 2025 trips with distance <= 1 mile:', len(november_2025_trips_leq_1_mile))

Date range: 2025-10-26 20:23:16 to 2025-12-01 20:29:00
Adjusted date range: 2025-11-01 00:00:13 to 2025-11-30 23:58:28
November 2025 trips with distance <= 1 mile: 8007


A: 8007 trips

#### Question 4

In [13]:
nov25_trips_less_100_miles = november_2025_trips[november_2025_trips['trip_distance'] < 100.0].copy()
nov25_trips_less_100_miles['pickup_date'] = nov25_trips_less_100_miles['lpep_pickup_datetime'].dt.date

nov25_trips_less_100_miles_grouped = nov25_trips_less_100_miles.loc[:,['pickup_date','trip_distance']] \
    .groupby('pickup_date').sum().sort_values(by='trip_distance', ascending=False)

nov25_trips_less_100_miles_grouped.iloc[0]

trip_distance    6377.03
Name: 2025-11-20, dtype: float64

**A: 2025-11-20 with 6377.03 miles**

#### Question 5

In [14]:
# Load zone_dataset

zone_dataset = pd.read_csv(zone_dataset_url)
zone_dataset.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [35]:
# Merge with zone dataset to get pickup zone names

pickup_zone_df = zone_dataset.add_prefix('pickup_')

nov25_trips_merged = november_2025_trips.merge(
    pickup_zone_df,
    left_on='PULocationID',
    right_on='pickup_LocationID',
    how='left',
)

# Filter to nov 18
nov25_18_trips_merged = nov25_trips_merged.loc[nov25_trips_merged['lpep_pickup_datetime'].dt.date == pd.to_datetime('2025-11-18').date()].copy()

# Get sum of total_amount by pickup zone
nov25_18_total_amount_by_pickup_zone = nov25_18_trips_merged.loc[:,['pickup_Zone', 'total_amount']]\
    .groupby('pickup_Zone').sum().sort_values(by='total_amount', ascending=False)

# Print top row
nov25_18_total_amount_by_pickup_zone.head(1)

,total_amount
pickup_Zone,
East Harlem North,9281.92


**A: East Harlem North with 9281.92$**

#### Question 6

In [36]:
# Add dropoff zones
dropoff_zone_df = zone_dataset.add_prefix('dropoff_')

nov25_trips_merged = nov25_trips_merged.merge(
    dropoff_zone_df,
    left_on='DOLocationID',
    right_on='dropoff_LocationID',
    how='left',
)

nov25_picked_up_east_harlem_north = nov25_trips_merged[nov25_trips_merged['pickup_Zone'] == 'East Harlem North']

nov25_picked_up_east_harlem_north_max_tip_by_dropoff_zone = nov25_picked_up_east_harlem_north.loc[:,['dropoff_Zone', 'tip_amount']] \
    .groupby('dropoff_Zone').max().sort_values(by='tip_amount', ascending=False)

nov25_picked_up_east_harlem_north_max_tip_by_dropoff_zone.head(1)

,tip_amount
dropoff_Zone,
Yorkville West,81.89


**A: Yorkville West had the largest tip with 81.89$**